In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
train_df

In [ ]:
test_df = pd.read_csv("test.csv", encoding="utf-8")
test_df

In [8]:
s = train_df.isna().sum()
# filter: 帶入跟資料大小依樣的true/false, true:留 false:滾
s[s > 0].sort_values(ascending=False)

Cabin       687
Age         177
Embarked      2
dtype: int64

In [9]:
s = test_df.isna().sum()
# filter: 帶入跟資料大小依樣的true/false, true:留 false:滾
s[s > 0].sort_values(ascending=False)

Cabin    327
Age       86
Fare       1
dtype: int64

In [12]:
# 數值類型: 中位數
med = train_df.median()
med = med.drop(["Pclass"])
train_df = train_df.fillna(med)
test_df = test_df.fillna(med)

In [17]:
# 類別類型: 最常出現
counts = train_df["Embarked"].value_counts()
most = counts.idxmax()
train_df["Embarked"] = train_df["Embarked"].fillna(most)